In [ ]:
######################################################
# get python version 
######################################################
from platform import python_version
python_version() 
# print(python_version())
######################################################
# Get GOOGLE API and TOKEN from environmnetal variable
######################################################
import os
CREDENTIALS = os.getenv('GOOGLE_CREDENTIALS')
TOKEN = os.environ.get('GOOGLE_TOKEN')

# write cedentials.json file
File = open("credentials.json", "w")
File.write(CREDENTIALS)
File.close()

# write token.json file
File = open("token.json", "w")
File.write(TOKEN)
File.close()

# print(CREDENTIALS)
# print(TOKEN)

In [ ]:
######################################################
# Import libraries
######################################################
import numpy as np
import pandas as pd;
from scipy import integrate
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from IPython.display import display
import ipywidgets as widgets
import IPython
import io
from googleapiclient.http import MediaIoBaseDownload
import shutil
import pathlib
from IPython.display import clear_output
import time
import scipy.io
import ipyspin


######################################################
# Handling some warning messages from matplotlib
######################################################
import warnings
warnings.filterwarnings("ignore")

import logging
logging.getLogger('matplotlib.font_manager').disabled = True

# make matplotlib to use the Rplot style sheet
plt.style.use('Rplot.mplstyle')
%matplotlib inline
plt.ioff()

# update matplotlib style 
plt.rcParams.update({'font.size': 18})
plt.rcParams['xtick.labelsize'] = 18 
plt.rcParams['ytick.labelsize'] = 18 
plt.rcParams['figure.titlesize'] = 20 

In [ ]:
######################################################
# Declare all global vra
######################################################
global Gdrive,Creds,Projects_List,Selected_Project,Project_Data_Files,Sensor_Layout_File,Pile_Properties_File, BaseLine_Data_File, Selected_Data_File, Time_Series_Figure_File
global Sensor_List, Pile_Properties, Figure, Figure_Axis, Color

######################################################
# Setting some state variables
######################################################
# data fike format
Data_File_Ext = ".mat";

# setting color list
Color = (plt.cm.tab20(np.linspace(0,1,16))) # r is 0 to 1 inclusive

# create ./temp directory
Temp_Directory = "./temp"
shutil.rmtree(Temp_Directory, ignore_errors=True)
os.mkdir(Temp_Directory)

In [ ]:
# extract date time string from Alicia_FileName
# filename = "20220803T200904LDC250_10550MHz_20ns_brillouin_peaks.mat"
# print(extract_date_time(filename))
def extract_date_time_string(Alicia_FileName):
    year  = Alicia_FileName[0:4];
    month = Alicia_FileName[4:6];
    day   = Alicia_FileName[6:8];
    hour  = Alicia_FileName[9:11];
    minutes  = Alicia_FileName[11:13];
    seconds  = Alicia_FileName[13:15];
    
    datetime_string = year+"/"+month+"/"+day+"-"+hour+":"+minutes+":"+seconds;
    return datetime_string

In [ ]:
######################################################
# Import Google API libraries
######################################################
from __future__ import print_function
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from getfilelistpy import getfilelist


# If modifying these scopes, delete the file token.json.
SCOPES = ['https://www.googleapis.com/auth/drive.metadata.readonly']
SCOPES = ['https://www.googleapis.com/auth/drive']

######################################################
# Connect GOOGLE DRIVE Remotely and save credentials
###################################################### 
def Connect_Google_Drive():
    """Shows basic usage of the Drive v3 API.
    Prints the names and ids of the first 10 files the user has access to.
    """
    
    global Gdrive,Creds;
    
    Creds = None
    Gdrive= None;
    # The file token.json stores the user's access and refresh tokens, and is
    # created automatically when the authorization flow completes for the first
    # time.
    if os.path.exists('token.json'):
        Creds = Credentials.from_authorized_user_file('token.json', SCOPES)
    # If there are no (valid) credentials available, let the user log in.
    if not Creds or not Creds.valid:
        if Creds and Creds.expired and Creds.refresh_token:
            Creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                'credentials.json', SCOPES)
            Creds = flow.run_local_server()
        # Save the credentials for the next run
        with open('token.json', 'w') as token:
            token.write(Creds.to_json())

    try:
        Gdrive = build('drive', 'v3', credentials=Creds)

    except HttpError as error:
        # TODO(developer) - Handle errors from drive API.
        print(f'An error occurred: {error}')

    return Gdrive, Creds

######################################################
# Get all the projects in folder "Pile_Load_Test"
# - Save all folder names in Projects_List 
# - Selected_Project is first project i.e. "1_Pratham_Test"
######################################################
def Get_Projects_List():
    
    global Creds,Projects_List,Selected_Project;
    
    #all pile load test files from google drive
    resource = {"oauth2":Creds,"id":'181AF6vL05nwQYW3i4uvwptF9huV3D1jt',"fields": "files(name)"}
    res = getfilelist.GetFileList(resource);
    Folders = res["folderTree"]["names"][1:]; # Pile load test folders
    Folders_Id = res["folderTree"]["folders"][1:]; # Pile load test folder ids
    
    Projects_List = [];
    for i in range(len(Folders)):
        Projects_List.append((Folders[i],Folders_Id[i]));
        
    Selected_Project = Projects_List[2]
        
    return Projects_List

######################################################
# Get all files from Selected_Project folder 
# -saves filenames and their respective ids in variables
######################################################
def Get_Project_Files():
    
    global Gdrive,Selected_Project,Project_Data_Files,Sensor_Layout_File, Time_Series_Figure_File, Pile_Properties_File, BaseLine_Data_File, Selected_Data_File;
    
    Project_Id = Selected_Project[1]; 
        
    #all pile load test files from google drive
    pageToken = ""
    files=[];
    while pageToken is not None:
        response=Gdrive.files().list(q="'{}' in parents and trashed=false".format(Project_Id), fields= 'nextPageToken, files(id,name)', orderBy="name", pageSize=900, pageToken=pageToken).execute();
        files.extend(response.get('files')); 
        pageToken = response.get('nextPageToken')
        
#     response=Gdrive.files().list(q="'{}' in parents and trashed=false".format(Project_Id), fields= 'files(id,name)', orderBy="name", pageSize=1000).execute();
#     files=(response.get('files')); 
        
#     print(files)
        
    Project_Data_Files = [];
    Sensor_Layout_File = None;
    Time_Series_Figure_File = None;
    Pile_Properties_File = None;
    
#     Print(files)
           
    for i in range(len(files)):
        fileName = files[i]["name"];
        fileId   = files[i]["id"];
        file_extension = pathlib.Path(fileName).suffix
        
#         print(fileName)
        
        if(fileName=="Pile_Properties.txt"):
            Pile_Properties_File = (fileName,fileId);
        elif (fileName=="Sensor_Layout.txt"):
            Sensor_Layout_File = (fileName,fileId);
        elif (fileName=="Time_Series_Plot.png"):
            Time_Series_Figure_File = (fileName,fileId);
        elif (file_extension==Data_File_Ext):
            Project_Data_Files.append((extract_date_time_string(fileName)+" PST",fileId));
        else:
            continue;
                    
    BaseLine_Data_File = ("BaseLine.dat",Project_Data_Files[0][1]);
    Selected_Data_File = ("SelectedData.dat",Project_Data_Files[-1][1]);
        
    Download_File(Pile_Properties_File)
    Download_File(Sensor_Layout_File)
    Download_File(Time_Series_Figure_File)
    Download_File(BaseLine_Data_File)
    Download_File(Selected_Data_File)
    Get_Pile_Properties();
    Get_Sensor_Layout();
    Process_Sensor_Data();
    
    return Project_Data_Files, Pile_Properties_File, Sensor_Layout_File, Time_Series_Figure_File

#########################################################################
# This function basically donwloads the files and saves it locally
def Download_File(FileName):
    
    global Gdrive;
    
    dirName = "./temp"
    if not os.path.exists(dirName):
        os.mkdir(dirName)  
    
    filepath= dirName+"/"+FileName[0];
    
    request = Gdrive.files().get_media(fileId=FileName[1])
    fh = io.BytesIO()
    
    downloader = MediaIoBaseDownload(fh, request)
    done = False
    while done is False:
        status, done = downloader.next_chunk()
        # print("Download %d%%" % int(status.progress() * 100))
        
    fh.seek(0)
    
    # Write the received data to the file
    with open(filepath, 'wb') as f:
        shutil.copyfileobj(fh, f)
        
# Connect_Google_Drive()
# print(Get_Projects_List())
# print(Selected_Project)
# # Selected_Project=('3_Oregon_Test_Pile_9403', '1Pnx_s7h48-E8xsA0ZS4ZNz4ds0MeZaww')
# # Selected_Project=('1GiFgRr4H8jEvYD1_99u5LO9bZ2tsPopB', '1GiFgRr4H8jEvYD1_99u5LO9bZ2tsPopB')
# print(Get_Project_Files())

In [ ]:
# Sensor_Class
class Sensor:
    
    def find_nearest(self,array, value):
        array = np.asarray(array)
        idx = (np.abs(array - value)).argmin()
        return idx,array[idx],np.abs(value-array[idx])

    def __init__(self,Name="S",Radius=1,Theta=0,Pile_Section=[0,1],Cable_Section=[0,1], Gage_Length=1, Type="strain"):
        self.Name             = Name;
        self.Radius           = Radius;
        self.Theta            = Theta;
        self.Pile_Section     = Pile_Section;
        self.Cable_Section    = Cable_Section;
        self.Pile_Coordinates = np.array([]);
        self.Cable_Coordinates= np.array([]);
        self.Gage_Length      = Gage_Length;
        self.Data             = np.array([]);
        self.Time             = np.array([]);
        self.Type             = Type;

    def __str__(self):
        Message = "";
        Message = Message + "Name:              " + str(self.Name) +             "\n";
        Message = Message + "Type:              " + str(self.Type) +             "\n";
        Message = Message + "Radius:            " + str(self.Radius) +           "\n";
        Message = Message + "Theta:             " + str(self.Theta) +            "\n";
        Message = Message + "Pile_Section:      " + str(self.Pile_Section) +     "\n";
        Message = Message + "Cable_Section:     " + str(self.Cable_Section) +    "\n";
        Message = Message + "Pile_Coordinates:  " + str(self.Pile_Coordinates) + "\n";
        Message = Message + "Cable_Coordinates: " + str(self.Cable_Coordinates) +"\n";
        Message = Message + "Gage_Length:       " + str(self.Gage_Length) +      "\n";
        Message = Message + "Time:              " + str(self.Time) +             "\n";
        Message = Message + "Data:              " + str(self.Data) +             "\n";

        return Message
    
    def Process_Data(self,Data,Time=None):
        Cable_Location         = np.linspace(0,(Data.shape[0]-1)*self.Gage_Length,Data.shape[0]);
        Cable_Start_Index,_,_  = self.find_nearest(Cable_Location,self.Cable_Section[0]);
        Cable_End_Index,_,_    = self.find_nearest(Cable_Location,self.Cable_Section[1]);
        self.Cable_Coordinates = Cable_Location[Cable_Start_Index:Cable_End_Index+1];
        self.Pile_Coordinates  = np.linspace(self.Pile_Section[0],self.Pile_Section[1],len(self.Cable_Coordinates));
        Pile_Section_Length    = (self.Pile_Section[1]-self.Pile_Section[0]);
        Cable_Section_Length   = (self.Cable_Section[1]-self.Cable_Section[0]);
        if(np.abs(np.round(Pile_Section_Length)) != np.abs(np.round(Cable_Section_Length))):
            print("\n\nWarning! Sensor " , self.Name, " has Pile Section Length (m) ", Pile_Section_Length , " different that Cable Section Length (m) ", Pile_Section_Length , "\n\n")
        self.Time              = Time;
        self.Data              = Data[Cable_Start_Index:Cable_End_Index+1];

# ################ Example ############################
# # note data should be arragend in (location x time)
# location = np.array([0,1,2,3,4]);
# data     = np.array([[2,1], [3,2], [4,4], [5,5], [6,6]])
# # data   = np.array([2,3,4,5,6])
    
# # pile section is r theta z 
# x = Sensor(Name="S",Radius=0.5, Theta = 0, Pile_Section=[0,2.1],Cable_Section=[1,3], Gage_Length=1, Type="temp");
# SelectedData = Read_Mat_Data("./temp/"+"SelectedData.dat");
# BaseLineData = Read_Mat_Data("./temp/"+"BaseLine.dat");

# # Sensor_Layout_Matrix = np.loadtxt("./temp/Sensor_Layout.txt", comments="#", delimiter=",", dtype=str);
# # print(Sensor_Layout_Matrix)
# print(x)
# x.Process_Data(data)
# print(x)

In [ ]:
# Get pile properties from file 'Pile_Properties.txt'
def Get_Pile_Properties():
    global Pile_Properties
    Pile_Properties_Matrix = np.loadtxt("./temp/Pile_Properties.txt", comments="#", delimiter=",", dtype=str);
    Diameter         = float(Pile_Properties_Matrix[0,1]);
    Density          = float(Pile_Properties_Matrix[1,1]);
    Elastic_Modulus  = float(Pile_Properties_Matrix[2,1]);
    Length           = float(Pile_Properties_Matrix[3,1]);
    Pile_Properties = [Diameter, Density, Elastic_Modulus, Length]
    return 

# Get Sensor layout from file 'Sensor_Layout.txt'
def Get_Sensor_Layout():
    global Sensor_List
    Sensor_Layout_Matrix = np.loadtxt("./temp/Sensor_Layout.txt", comments="#", delimiter=",", dtype=str);
    gage_length          = float(Sensor_Layout_Matrix[0,1]);
    Num_of_Sensors       = Sensor_Layout_Matrix.shape[0]-1;
    Sensor_List          = np.empty((Num_of_Sensors),dtype=Sensor);
    for i in range(1,Num_of_Sensors+1):
        name   = Sensor_Layout_Matrix[i,0];
        radius = float(Sensor_Layout_Matrix[i,1]);
        theta  = float(Sensor_Layout_Matrix[i,2]);
        pile_section = np.array([float(Sensor_Layout_Matrix[i,3]),float(Sensor_Layout_Matrix[i,4])]);
        cable_section = np.array([float(Sensor_Layout_Matrix[i,5]),float(Sensor_Layout_Matrix[i,6])]);
        sensor_type = Sensor_Layout_Matrix[i,7];
        if(sensor_type=="strain" or sensor_type=="s"):
            sensor_type="strain";
        else:
            sensor_type="temp"
        Sensor_List[i-1]=Sensor(name, radius, theta, pile_section, cable_section, gage_length,sensor_type)
    return

def Read_Mat_Data(FileName):
    f      = scipy.io.loadmat(FileName)
    # 1 micro strain = .04998 MHz
    strain = f["brillouin_peaks"][0]/(0.04998*10**6);
    return strain

def Process_Sensor_Data():
    global Sensor_List,BaseLine_Data_File, Selected_Data_File;
    
    SelectedData = Read_Mat_Data("./temp/"+Selected_Data_File[0]);
    BaseLineData = Read_Mat_Data("./temp/"+BaseLine_Data_File[0]);
        
    for i in range(0,len(Sensor_List)):        
        Sensor_List[i].Process_Data(SelectedData-BaseLineData);  

Connect_Google_Drive();# connect to Google Drive 
Get_Projects_List();   # get project list 
Get_Project_Files();   # get project files

# print(Projects_List)
# print(Project_Data_Files)

In [ ]:
# Connect_Google_Drive();# connect to Google Drive 
# Get_Projects_List();   # get project list 
# Get_Project_Files();   # get project files

def Draw_Pile_Crossection_with_Sensors(Strain_Sensor_Axis,Temp_Sensor_Axis):
    
    global Sensor_List, Color, Pile_Properties;
    
    # create a pile crossection
    Diameter = 1.0;
    Pile_Crossection_1 = plt.Circle((0,0), Diameter/2, facecolor="gray",edgecolor="k",linewidth=2,linestyle="-");
    Pile_Crossection_2 = plt.Circle((0,0), Diameter/2, facecolor="gray",edgecolor="k",linewidth=2,linestyle="-");
    Strain_Sensor_Axis.add_artist(Pile_Crossection_1)
    Temp_Sensor_Axis.add_artist(Pile_Crossection_2)
    
    for i in range(len(Sensor_List)):
        Sensor_type = Sensor_List[i].Type;
        Sensor_Dia = 8/100*Diameter;
        theta = np.radians(Sensor_List[i].Theta);
        x = (Sensor_List[i].Radius/Pile_Properties[0])*np.cos(theta);
        y = (Sensor_List[i].Radius/Pile_Properties[0])*np.sin(theta);
        Sensor_plot = plt.Circle((x,y), Sensor_Dia/2, facecolor=Color[i],edgecolor="k",linewidth=1,linestyle="-");
        
        Crossection_Axis=Strain_Sensor_Axis;
        if(Sensor_type=="temp"):
            Crossection_Axis = Temp_Sensor_Axis;
            
        Crossection_Axis.text(x*1.25,y*1.25," "+Sensor_List[i].Name+" ", fontsize = 12, rotation=30+theta, horizontalalignment='center', verticalalignment='center')
        Crossection_Axis.add_artist(Sensor_plot);

    Strain_Sensor_Axis.plot([-Diameter*0.75,Diameter*0.75],[0,0],"--y",linewidth=2);
    Strain_Sensor_Axis.set_xlim([-0.75*Diameter, 0.75*Diameter]);
    Strain_Sensor_Axis.set_ylim([-0.75*Diameter, 0.75*Diameter]);
    Strain_Sensor_Axis.axis('off');plt.grid(b=None);
    Strain_Sensor_Axis.title.set_text('Strain Sensors \n');

    Temp_Sensor_Axis.plot([-Diameter*0.75,Diameter*0.75],[0,0],"--y",linewidth=2);
    Temp_Sensor_Axis.set_xlim([-0.75*Diameter, 0.75*Diameter]);
    Temp_Sensor_Axis.set_ylim([-0.75*Diameter, 0.75*Diameter]);
    Temp_Sensor_Axis.axis('off');plt.grid(b=None);
    Temp_Sensor_Axis.title.set_text('Temperature Sensors \n');
    
    return

def Draw_Pile_Properties(Pile_Axis):
    
    global Pile_Properties;
        
    # create a pile crossection
    Pile_Axis.set_xlim([-1, 1]);
    Pile_Axis.set_ylim([-1, 1]);
    Pile_Axis.axis('off');Pile_Axis.grid(b=None)
    
    Pile_Axis.text(-0.9,0.75, " Diameter (m) " + str(Pile_Properties[0]), fontsize = 12, horizontalalignment='left', verticalalignment='center');
    Pile_Axis.text(-0.9,0.25, " Density (kg/m^3) " + str(Pile_Properties[1]), fontsize = 12, horizontalalignment='left', verticalalignment='center');
    Pile_Axis.text(-0.9,-0.25," Elastic Modulus (GPa) " + str(Pile_Properties[2]), fontsize = 12, horizontalalignment='left', verticalalignment='center');   
    Pile_Axis.text(-0.9,-0.75," Length (m) " + str(Pile_Properties[3]), fontsize = 12, horizontalalignment='left', verticalalignment='center');   
    Pile_Axis.title.set_text('Pile Properties \n')

    return

def Plot_Sensor_Data():

    global Sensor_List, Pile_Properties, Figure_Axis, Figure, Color;
    from scipy import integrate

#     clear_output(wait=True)
        
    ###### Figure Widget #########################
    Figure             = plt.figure(figsize=(36,12));
    Grid_Space         = gridspec.GridSpec(3, 5, width_ratios=[2,2,2,2,1], height_ratios=[1,1,1]);
    Total_Strain_Axis  = Figure.add_subplot(Grid_Space[:,0]);
    Temp_Axis          = Figure.add_subplot(Grid_Space[:,1]);
    Mec_Strain_Axis    = Figure.add_subplot(Grid_Space[:,2]);
    Settlement_Axis    = Figure.add_subplot(Grid_Space[:,3]);
    Strain_Sensor_Axis = Figure.add_subplot(Grid_Space[0,4]);
    Temp_Sensor_Axis   = Figure.add_subplot(Grid_Space[1,4]);
    Pile_Prop_Axis     = Figure.add_subplot(Grid_Space[2,4]);
     
    Draw_Pile_Crossection_with_Sensors(Strain_Sensor_Axis,Temp_Sensor_Axis);
    Draw_Pile_Properties(Pile_Prop_Axis);
    
    min_z = 0;
    max_z = 0;
          
    for i in range(len(Sensor_List)):
        Sensor_type = Sensor_List[i].Type;
        loc         = Sensor_List[i].Pile_Coordinates;
        strain      = Sensor_List[i].Data;
        
        min_z = min(loc[0],min_z);
        max_z = max(loc[-1],max_z);
        
        if(Sensor_type=="strain"):
            Total_Strain_Axis.plot(strain,loc,label=Sensor_List[i].Name,linewidth=2, color=Color[i]);
            Total_Strain_Axis.set_xlim([0,2000]) 
        else:
            Temp_Axis.plot(strain*(0.050/1.0),loc,label=Sensor_List[i].Name,linewidth=2, color=Color[i]);
#             Temp_Axis.plot(strain,loc,label=Sensor_List[i].Name,linewidth=2, color=Color[i]); 
            Temp_Axis.set_xlim([0,70])
   
    once = True;
    for i in range(len(Sensor_List)):
        if(Sensor_List[i].Type=="strain"):
#             total_strain     = Sensor_List[i].Data;
            total_strain_loc = Sensor_List[i].Pile_Coordinates;
            sorted_index = np.argsort(total_strain_loc)
        
            total_strain = Sensor_List[i].Data[sorted_index]
            total_strain_loc = total_strain_loc[sorted_index]
            
            strain_sensor_name = Sensor_List[i].Name;
            
            # find corresponding temp sensor 
            for j in range(len(Sensor_List)):
                if(Sensor_List[j].Type=="temp" and Sensor_List[j].Name == "T"+strain_sensor_name[1]):
                    break;
                       
            temp_strain_loc = Sensor_List[j].Pile_Coordinates;
            sorted_index = np.argsort(temp_strain_loc)
        
            temp_strain = Sensor_List[j].Data[sorted_index]
            temp_strain_loc = temp_strain_loc[sorted_index]
            
            
            loc_start = max(total_strain_loc[0],temp_strain_loc[0]);
            loc_end   = min(total_strain_loc[-1],temp_strain_loc[-1]);
            
            loc = np.linspace(loc_start,loc_end,2*max(len(temp_strain_loc),len(total_strain_loc)));
            
            temp_strain  = np.interp(loc,temp_strain_loc,temp_strain)/1.0*1.0;
            total_strain = np.interp(loc,total_strain_loc,total_strain);
            
            mec_strain  = total_strain-temp_strain;
            Mec_Strain_Axis.plot(mec_strain,loc,'--',linewidth=2, color=Color[i]);
            
            mec_settlement = np.flip(integrate.cumulative_trapezoid(mec_strain,dx=np.abs(loc[1]-loc[0]),initial=0))*10**-3;
            tot_settlement = np.flip(integrate.cumulative_trapezoid(total_strain,dx=np.abs(loc[1]-loc[0]),initial=0))*10**-3;
            
#             if(once):
#                 Settlement_Axis.plot(-mec_settlement,loc,'--',label="mech strain",linewidth=2, color=Color[i]);
#                 Settlement_Axis.plot(-tot_settlement,loc,label="total strain",linewidth=2, color=Color[i]);
#                 once=False;
#             else:
#                 Settlement_Axis.plot(-mec_settlement,loc,'--',linewidth=2, color=Color[i]);
#                 Settlement_Axis.plot(-tot_settlement,loc,linewidth=2, color=Color[i]);
            Settlement_Axis.plot(-mec_settlement,loc,'--',linewidth=2, color=Color[i]);
    
    Total_Strain_Axis.set_xlabel("Total Strain ($\mu \epsilon$)"); Total_Strain_Axis.set_ylabel("Depth (m)");     Total_Strain_Axis.set_ylim([max_z,min_z]);
    Temp_Axis.set_xlabel("Temp ($^o C$)");             Temp_Axis.set_ylabel("Depth (m)");       Temp_Axis.set_ylim([max_z,min_z]);
    Mec_Strain_Axis.set_xlabel("Mec Strain ($\mu \epsilon$)"); Mec_Strain_Axis.set_ylabel("Depth (m)");     Mec_Strain_Axis.set_ylim([max_z,min_z]);
    Settlement_Axis.set_xlabel("Displacement (mm)");     Settlement_Axis.set_ylabel("Depth (m)"); Settlement_Axis.set_ylim([max_z,min_z]);
    
    handles_strain, labels_strain = Total_Strain_Axis.get_legend_handles_labels();
    handles_temp, labels_temp = Temp_Axis.get_legend_handles_labels();
    handles_Settlement, labels_Settlement = Settlement_Axis.get_legend_handles_labels();

    handles,labels = [],[];
    
    for h in handles_strain:
        handles.append(h)
        
    for h in handles_temp:
        handles.append(h)

    for h in handles_Settlement:
        handles.append(h)
        
    for l in labels_strain:
        labels.append(l)
        
    for l in labels_temp:
        labels.append(l)
        
    for l in labels_Settlement:
        labels.append(l)   
        
    Figure.legend(handles, labels, loc="upper center",borderaxespad=.5, ncol=12,fontsize = 'small');   
#     Total_Strain_Axis.legend(bbox_to_anchor=(0,1.02,1,0.2), loc="lower left",mode="expand", borderaxespad=0, ncol=5,fontsize = 'small');

    warnings.filterwarnings("ignore")

    Figure.tight_layout();
    Figure.savefig("./temp/SensorPlot.png");
    Figure.clf();
    return;

# Plot_Sensor_Data()
# plt.show()

In [ ]:
# ###### GageLength Widget #########################
# Gage_Length_Widget = widgets.Dropdown(
#                   options=["0.0026","0.25","0.5", "1", "2", "3"],
#                   value=str(Gage_Length),
#                   description='<font size="3"> <b> Gage (m) : </b></font>',
#                   disabled=False,);

# def Gage_Length_Widget_Change(change):
#     Spinner_Widget.spin()
#     if change['type'] == 'change' and change['name'] == 'value':
#         global Gage_Length, Selected_Project,Sensor_Layout_File,Pile_Properties_File,BaseLine_Data_File,Selected_Data_File;
        
#         Gage_Length = float(Gage_Length_Widget.value)
#         Process_Sensor_Data();
#         Plot_Sensor_Data();
#         SensorPlot_Widget.value=open("./temp/SensorPlot.png", "rb").read();
#     Spinner_Widget.stop()
        
# Gage_Length_Widget.observe(Gage_Length_Widget_Change);
# ###############################################

################### Spinner Widget ##############
Spinner_Widget = ipyspin.Spinner()
Spinner_Widget.layout.height = '0px'
Spinner_Widget.layout.width = '0px'
Spinner_Widget.lines = 10  # The number of lines to draw
Spinner_Widget.length = 5  # The length of each line
Spinner_Widget.width = 5  # The line thickness
Spinner_Widget.radius = .5  # The radius of the inner circle
Spinner_Widget.stop()

In [ ]:
###### Projects Widget #########################
Projects_Widget = widgets.Dropdown(
                  options=Projects_List,
                  value=Selected_Project[1],
                  description='<font size="3"> <b> Projects: </b></font>',
                  disabled=True,);

def Projects_Widget_Change(change):
    Spinner_Widget.layout.height = '25px'
    Spinner_Widget.layout.width = '25px'
    Spinner_Widget.spin()
    Update_Label_Widget.value="Updating...... Please Wait";
    
    if change['type'] == 'change' and change['name'] == 'value':
        global Selected_Project,Sensor_Layout_File,Time_Series_Figure_File, Pile_Properties_File,BaseLine_Data_File,Selected_Data_File;
        
        Selected_Project = (Projects_Widget.label,Projects_Widget.value)
        Get_Project_Files();
        BaseLine_Widget.options = Project_Data_Files;
        SelectedData_Widget.options = Project_Data_Files;
        BaseLine_Widget.value   = Project_Data_Files[0][1];
        SelectedData_Widget.value   = Project_Data_Files[-1][1];  
        Download_File(Pile_Properties_File);
        Download_File(Sensor_Layout_File);
        Download_File(Time_Series_Figure_File);
        Download_File(BaseLine_Data_File);
        Download_File(Selected_Data_File);
        Get_Pile_Properties();
        Get_Sensor_Layout();
        Process_Sensor_Data();
        Plot_Sensor_Data();
        SensorPlot_Widget.value=open("./temp/SensorPlot.png", "rb").read();
        Time_Series_SensorPlot_Widget.value=open("./temp/Time_Series_Plot.png", "rb").read();
#         print(Project_Data_Files)

    Spinner_Widget.stop()
    Update_Label_Widget.value="";

        
Projects_Widget.observe(Projects_Widget_Change);
###############################################
      
###### BaseLine Widget #########################
BaseLine_Widget = widgets.Dropdown(
                  options=Project_Data_Files,
                  value=Project_Data_Files[0][1],
                  description='<font size="3"> <b> Baseline: </b></font>',
                  disabled=True,);

def BaseLine_Widget_Change(change):
    Spinner_Widget.layout.height = '25px'
    Spinner_Widget.layout.width = '25px'
    Spinner_Widget.spin()
    Update_Label_Widget.value="Updating...... Please Wait";
    
    if change['type'] == 'change' and change['name'] == 'value':
        global BaseLine_Data_File, Time_Series_Figure_File;
        BaseLine_Data_File = ("BaseLine.dat",BaseLine_Widget.value);
        Download_File(BaseLine_Data_File);
        Download_File(Time_Series_Figure_File);
        Process_Sensor_Data();
        Plot_Sensor_Data();
        SensorPlot_Widget.value=open("./temp/SensorPlot.png", "rb").read();
        Time_Series_SensorPlot_Widget.value=open("./temp/Time_Series_Plot.png", "rb").read();
   
    Spinner_Widget.stop()
    Update_Label_Widget.value="";


BaseLine_Widget.observe(BaseLine_Widget_Change);
###############################################

###### DataFile Widget #########################
SelectedData_Widget = widgets.Dropdown(
                options=Project_Data_Files,
                value=Project_Data_Files[-1][1],
                description='<font size="3"> <b> Data: </b></font>',
                disabled=False,);

def SelectedData_Widget_Change(change):
    Spinner_Widget.layout.height = '25px'
    Spinner_Widget.layout.width = '25px'
    Spinner_Widget.spin()
    Update_Label_Widget.value="Updating...... Please Wait";
    
    if change['type'] == 'change' and change['name'] == 'value':
        global Selected_Data_File, Time_Series_Figure_File;
        Selected_Data_File = ("SelectedData.dat",SelectedData_Widget.value);
        Download_File(Selected_Data_File);
        Download_File(Time_Series_Figure_File)
        Process_Sensor_Data();
        Plot_Sensor_Data();
        SensorPlot_Widget.value=open("./temp/SensorPlot.png", "rb").read();
        Time_Series_SensorPlot_Widget.value=open("./temp/Time_Series_Plot.png", "rb").read();
    
    Spinner_Widget.stop()
    Update_Label_Widget.value="";


SelectedData_Widget.observe(SelectedData_Widget_Change);
###############################################

## Plot Sensor Data
Plot_Sensor_Data();

###### Image Widget #########################
SensorPlot_Widget = widgets.Image(
                            value=open("./temp/SensorPlot.png", "rb").read(),
                            format='png',
                            width=20000,
                            height=8000,
                        );
###############################################

###### Image Widget #########################
Time_Series_SensorPlot_Widget = widgets.Image(
                            value=open("./temp/Time_Series_Plot.png", "rb").read(),
                            format='png',
                            width=20000,
                            height=8000,
                        );
###############################################

###### Sync Widget #########################
Synchronize_Widget = widgets.Button(
                    description='Update',
                    disabled=False,
                    button_style='', # 'success', 'info', 'warning', 'danger' or ''
                    tooltip='Synchronize',
                    icon='' # (FontAwesome names without the `fa-` prefix)
                );

def Synchronize_Widget_clicked(button):
#     Spinner_Widget.layout.height = '25px'
#     Spinner_Widget.layout.width = '25px'
#     Spinner_Widget.spin()
#     Get_Project_Files();
#     BaseLine_Widget.options=Project_Data_Files;
#     SelectedData_Widget.options=Project_Data_Files;
#     SelectedData_Widget.value  =Project_Data_Files[-1][1];
#     button.button_style="success";
#     time.sleep(1);
#     button.button_style="";
#     Spinner_Widget.stop()
    
    Spinner_Widget.layout.height = '25px'
    Spinner_Widget.layout.width = '25px'
    Spinner_Widget.spin()
    Update_Label_Widget.value="Updating...... Please Wait";
    
    Selected_Project = (Projects_Widget.label,Projects_Widget.value)
    Get_Project_Files();
    BaseLine_Widget.options     = Project_Data_Files;
    SelectedData_Widget.options = Project_Data_Files;
    BaseLine_Widget.value       = Project_Data_Files[0][1];
    SelectedData_Widget.value   = Project_Data_Files[-1][1];  
    Download_File(Pile_Properties_File);
    Download_File(Sensor_Layout_File);
    Download_File(Time_Series_Figure_File);
    Download_File(BaseLine_Data_File);
    Download_File(Selected_Data_File);
    Get_Pile_Properties();
    Get_Sensor_Layout();
    Process_Sensor_Data();
    Plot_Sensor_Data();
    SensorPlot_Widget.value=open("./temp/SensorPlot.png", "rb").read();
    Time_Series_SensorPlot_Widget.value=open("./temp/Time_Series_Plot.png", "rb").read();
        
    Spinner_Widget.stop()
    Update_Label_Widget.value="";
    
Synchronize_Widget.on_click(Synchronize_Widget_clicked)
###############################################

###### Label Widget #############################
Update_Label_Widget = widgets.Label(value="")


In [ ]:
# import asyncio
def func2():  
    Spinner_Widget.layout.height = '25px'
    Spinner_Widget.layout.width = '25px'
    Spinner_Widget.spin()
    Update_Label_Widget.value="Updating...... Please Wait";
    
    Selected_Project = (Projects_Widget.label,Projects_Widget.value)
    Get_Project_Files();
    BaseLine_Widget.options     = Project_Data_Files;
    SelectedData_Widget.options = Project_Data_Files;
    BaseLine_Widget.value       = Project_Data_Files[0][1];
    SelectedData_Widget.value   = Project_Data_Files[-1][1];  
    Download_File(Pile_Properties_File);
    Download_File(Sensor_Layout_File);
    Download_File(Time_Series_Figure_File);
    Download_File(BaseLine_Data_File);
    Download_File(Selected_Data_File);
    Get_Pile_Properties();
    Get_Sensor_Layout();
    Process_Sensor_Data();
    Plot_Sensor_Data();
    SensorPlot_Widget.value=open("./temp/SensorPlot.png", "rb").read();
    Time_Series_SensorPlot_Widget.value=open("./temp/Time_Series_Plot.png", "rb").read();
        
    Spinner_Widget.stop()
    Update_Label_Widget.value="";
#     await asyncio.sleep(2*60)
    



<center> <font size="6"> <b> Pile DSS Analyzer </b></font> <a href="https://github.com/SumeetSinha/Pile_DSS_Analyzer#readme"  target="_blank">usage</a>
</center> 

In [ ]:
display(widgets.HBox((Projects_Widget,BaseLine_Widget,SelectedData_Widget,Spinner_Widget,Synchronize_Widget,Update_Label_Widget)))
display(SensorPlot_Widget)
display(Time_Series_SensorPlot_Widget)
# func2()